In [ ]:
import asyncio
import aiohttp
import json
from datetime import datetime
import os
from google.colab import files

class RealKudaGoClient:
    """Реальный клиент для KudaGo API"""

    def __init__(self):
        self.base_url = "https://kudago.com/public-api/v1.4"

    async def fetch_real_events(self):
        """Получение реальных данных с KudaGo"""
        print(" Подключаемся к реальному API KudaGo...")

        # Категории, которые точно работают (проверено)
        working_categories = {
            'concert': 'Концерты',
            'exhibition': 'Выставки',
            'theater': 'Театр',
            'festival': 'Фестивали'
        }

        all_events = []

        async with aiohttp.ClientSession() as session:
            for cat_code, cat_name in working_categories.items():
                print(f"📡 Загружаем {cat_name}...")

                params = {
                    'categories': cat_code,
                    'location': 'spb',
                    'page_size': 50,
                    'actual_since': int(datetime.now().timestamp()),
                    'fields': 'id,title,description,dates,price,place,images,site_url,tags,age_restriction'
                }

                try:
                    async with session.get(
                        f"{self.base_url}/events/",
                        params=params,
                        timeout=aiohttp.ClientTimeout(total=30)
                    ) as response:

                        if response.status == 200:
                            data = await response.json()
                            events = data.get('results', [])

                            for event in events:
                                # Проверяем, что событие имеет необходимые поля
                                if event.get('title') and event.get('dates'):
                                    processed_event = {
                                        'id': event.get('id'),
                                        'title': event.get('title'),
                                        'description': event.get('description', ''),
                                        'category': cat_code,
                                        'category_name': cat_name,
                                        'dates': event.get('dates', []),
                                        'price': event.get('price', 'Не указана'),
                                        'place': event.get('place', {}),
                                        'url': event.get('site_url', ''),
                                        'tags': event.get('tags', []),
                                        'images': event.get('images', []),
                                        'age_restriction': event.get('age_restriction'),
                                        'source': 'kudago',
                                        'parsed_at': datetime.now().isoformat(),
                                        'is_real_data': True  # Флаг реальных данных
                                    }
                                    all_events.append(processed_event)

                            print(f"  Получено {len(events)} событий, обработано {len([e for e in all_events if e['category'] == cat_code])}")
                        else:
                            print(f"   Ошибка {response.status}")

                except Exception as e:
                    print(f"   Исключение: {type(e).__name__}")

                # Задержка между запросами
                await asyncio.sleep(1)

        return all_events

async def get_real_data():
    """Получение реальных данных"""
    client = RealKudaGoClient()
    real_events = await client.fetch_real_events()
    return real_events

# Запускаем сбор реальных данных
print("="*60)
print(" ЗАГРУЗКА РЕАЛЬНЫХ ДАННЫХ ИЗ KUDAGO API")
print("="*60)

real_events = await get_real_data()

if real_events:
    print(f"\n УСПЕХ! Получено {len(real_events)} реальных событий")

    # Сохраняем реальные данные
    data_dir = '/content/data/real'
    os.makedirs(data_dir, exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    real_file = f'{data_dir}/spb_real_events_{timestamp}.json'

    metadata = {
        'city': 'spb',
        'city_name': 'Санкт-Петербург',
        'parsed_at': datetime.now().isoformat(),
        'total_events': len(real_events),
        'categories_count': len(set(e['category'] for e in real_events)),
        'source': 'kudago',
        'is_real_data': True,
        'api_status': 'success',
        'note': 'Реальные данные, полученные через KudaGo API'
    }

    full_data = {
        'metadata': metadata,
        'events': real_events
    }

    with open(real_file, 'w', encoding='utf-8') as f:
        json.dump(full_data, f, ensure_ascii=False, indent=2)

    print(f"\n СТАТИСТИКА РЕАЛЬНЫХ ДАННЫХ:")
    print("-" * 40)

    # Анализ реальных данных
    categories = {}
    for event in real_events:
        cat = event['category_name']
        categories[cat] = categories.get(cat, 0) + 1

    print(f"  Город: Санкт-Петербург")
    print(f" Всего событий: {len(real_events)}")
    print(f"  Категорий: {len(categories)}")

    print(f"\n Распределение по категориям:")
    for cat, count in categories.items():
        print(f"  {cat}: {count} событий")

    # Показываем примеры реальных событий
    print(f"\n ПРИМЕРЫ РЕАЛЬНЫХ СОБЫТИЙ:")
    print("-" * 40)

    for i, event in enumerate(real_events[:3]):
        print(f"\n{i+1}. {event['title']}")
        print(f"   Категория: {event['category_name']}")
        print(f"   Цена: {event['price']}")
        if event.get('place') and event['place'].get('title'):
            print(f"   Место: {event['place']['title']}")
        if event.get('dates'):
            # Преобразуем timestamp в читаемую дату
            try:
                date_info = event['dates'][0]
                if date_info.get('start'):
                    dt = datetime.fromtimestamp(date_info['start'])
                    print(f"   Дата: {dt.strftime('%d.%m.%Y %H:%M')}")
            except:
                pass

    # Скачиваем файл
    print(f"\nСкачиваем файл с реальными данными...")
    files.download(real_file)

    print(f"\n РЕАЛЬНЫЕ ДАННЫЕ УСПЕШНО СОХРАНЕНЫ!")
    print(f" Файл: {real_file}")

else:
    print("\n Не удалось получить реальные данные")

    # Если API не работает, давайте попробуем альтернативный источник
    print("\n Пробуем альтернативный источник...")

    # Используем TimePad API как альтернативу
    import requests

    try:
        print(" Подключаемся к TimePad API...")
        response = requests.get(
            'https://api.timepad.ru/v1/events',
            params={
                'limit': 50,
                'skip': 0,
                'cities': 'spb',
                'sort': '+starts_at',
                'fields': 'name,description_short,starts_at,ends_at,location,poster_image,url'
            },
            timeout=10
        )

        if response.status_code == 200:
            data = response.json()
            timepad_events = []

            for item in data.get('values', []):
                timepad_event = {
                    'id': item.get('id'),
                    'title': item.get('name', ''),
                    'description': item.get('description_short', ''),
                    'category': 'event',
                    'category_name': 'Событие',
                    'dates': [{
                        'start': item.get('starts_at'),
                        'end': item.get('ends_at')
                    }],
                    'price': 'Не указана',
                    'place': {
                        'title': item.get('location', {}).get('city', 'СПб'),
                        'address': item.get('location', {}).get('address', '')
                    },
                    'url': item.get('url', ''),
                    'source': 'timepad',
                    'is_real_data': True
                }
                timepad_events.append(timepad_event)

            print(f" Получено {len(timepad_events)} событий из TimePad")

            # Сохраняем
            data_dir = '/content/data/real'
            os.makedirs(data_dir, exist_ok=True)

            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            timepad_file = f'{data_dir}/spb_timepad_events_{timestamp}.json'

            metadata = {
                'city': 'spb',
                'source': 'timepad',
                'parsed_at': datetime.now().isoformat(),
                'total_events': len(timepad_events),
                'api_status': 'success',
                'note': 'Реальные данные из TimePad API'
            }

            full_data = {
                'metadata': metadata,
                'events': timepad_events
            }

            with open(timepad_file, 'w', encoding='utf-8') as f:
                json.dump(full_data, f, ensure_ascii=False, indent=2)

            print(f"\n Скачиваем данные из TimePad...")
            files.download(timepad_file)

        else:
            print(f" TimePad API не доступен: {response.status_code}")

    except Exception as e:
        print(f"Ошибка при подключении к TimePad: {e}")

🚀 ЗАГРУЗКА РЕАЛЬНЫХ ДАННЫХ ИЗ KUDAGO API
🔗 Подключаемся к реальному API KudaGo...
📡 Загружаем Концерты...
  ✅ Получено 50 событий, обработано 50
📡 Загружаем Выставки...
  ✅ Получено 50 событий, обработано 50
📡 Загружаем Театр...
  ✅ Получено 50 событий, обработано 50
📡 Загружаем Фестивали...
  ✅ Получено 16 событий, обработано 16

✅ УСПЕХ! Получено 166 реальных событий

📊 СТАТИСТИКА РЕАЛЬНЫХ ДАННЫХ:
----------------------------------------
🏙️  Город: Санкт-Петербург
🎯 Всего событий: 166
🏷️  Категорий: 4

📈 Распределение по категориям:
  Концерты: 50 событий
  Выставки: 50 событий
  Театр: 50 событий
  Фестивали: 16 событий

🎯 ПРИМЕРЫ РЕАЛЬНЫХ СОБЫТИЙ:
----------------------------------------

1. музыкальный проект #В_СВЕЧАХ в особняках и дворцах Петербурга
   Категория: Концерты
   Цена: от 2500 до 6250 рублей
   Дата: 06.07.2023 17:00

2. кабаре-бурлеск-шоу Madam Boom
   Категория: Концерты
   Цена: от 1500 до 3000 руб.
   Дата: 21.11.2025 17:00

3. концерты органной и старинной музык

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ РЕАЛЬНЫЕ ДАННЫЕ УСПЕШНО СОХРАНЕНЫ!
📁 Файл: /content/data/real/spb_real_events_20251205_143858.json
